## A. What is it

---

**Typy** is a light-weight python library which gives some strong type support for Python 2.7 when you operate with variables, classes and functions.

* minimal change of code
* auto type check of variables
* strong type feature and weak type feature can co-exist

Requires:

* python >= 2.7

Platforms:

* Linux
* Mac
* Windows

Author:

Ray (Hou Songlin)
rock307977586@gmail.com

## B.Why to use?

---

What problem can happen with weak typed classes in python?

### 1.First we define a normal class

In [1]:
class Employee(object):
    
    total_number = 0
    
    def __init__(self,name,sex,age,salary):
        self.name = name # we hope it would be a str
        self.sex = sex # str can do
        self.age = age # int is fine
        self.salary = salary # int
        Employee.total_number += 1
        
    def greet(self,words):
        print "{} says: {}".format(self.name,words)

### 2. It is likely the improper types are alwayes used when we create classes

In [2]:
jack = Employee('jack','male',26,3000) # correct
mary = Employee('mary','female',28,[3500]) # incorrect but this passes test
bob = Employee(0,1,'34','3400') # incorrect but this passes test

### 3. When calling methods, sometimes type mismatch can bring misunderstanding

In [3]:
jack.greet("hello") # a meaningful parameter
mary.greet(1522312) # an unmeaningful parameter, but it can be executed without errors
bob.greet("morning") # information about bob is not correct, but it still passes.

jack says: hello
mary says: 1522312
0 says: morning


### 4. Problems can easily occur

In [4]:
total_salary = jack.salary + mary.salary + bob.salary
print "total salary is {}".format(total_salary)

TypeError: unsupported operand type(s) for +: 'int' and 'list'

## C. How to use?

In [5]:
from typy.typy import TypeBase
from typy.type_decorator import Typed

In [6]:
class Employee(TypeBase):
    
    total_number = 0
    
    @Typed # put a decorator here
    def __init__(self,name,sex,age,salary,types={'name':str,'sex':str,
                                                 'age':int,'salary':int}): # type declaration
        #we can safely assume they are of proper types now
        self.name = name 
        self.sex = sex 
        self.age = age 
        self.salary = salary 
        Employee.total_number += 1
       
    @Typed # put a decorator here
    def greet(self,words,types={'words':str,'return':bool}): # type declaration, 'return' means the type of returned value 
        print "{} says: {}".format(self.name,words)
        return True # this must be a bool value because we specified the 'return' type

### 1. Let's retest the code above

In [7]:
jack = Employee('jack','male',26,3000) # correct, so it passes

In [8]:
mary = Employee('mary','female',28,[3500]) # incorrect, so there is an error

AttributeError: expecting salary to be of <type 'int'>, but got <type 'list'>

In [9]:
bob = Employee(0,1,'34','3400') # incorrect, so there is an error

AttributeError: expecting name to be of <type 'str'>, but got <type 'int'>

In [10]:
jack.greet("hello") # runs like normal

jack says: hello


True

### 2. Go deeper, let's see variables outside classes

Since there are many limitations in implementing strong typed variables, we have to change a little bit to achieve our strong type goal.

In [11]:
from typy.typy import TypeVar

There are two ways of using typed variables. Usually it behaves like we didn't use types

In [12]:
name1 = TypeVar(str,"jack") # explicitly declare type
name2 = TypeVar.var("bob") # type inference

In [13]:
name1,name2

(jack, bob)

In [14]:
name1 == 'jack'

True

In [15]:
print name1 + " and " +  name2

jack and bob


In [16]:
TypeVar.var(3) + 2

5

Sometimes we have to refer to their original values. We use **v** attribute.

In [17]:
typed_var = TypeVar.var(3)
2 ** typed_var

TypeError: unsupported operand type(s) for ** or pow(): 'int' and 'TypeVar'

In [18]:
2 ** typed_var.v # typed_var.v refers to the original value of typed_var

8

Due to python limitation, the assignment consistency cannot be achieved using equal sign, so we should a method to assign values.

In [19]:
typed_var.t # this is an int value

Type wrapper for <type 'int'>

In [20]:
typed_var.t_raw # this can return 'int' similar to the built-in type() method

int

In [21]:
typed_var.assign(8) # this will pass

In [22]:
typed_var.assign('6') # this won't

TypeError: Assignment failed. expecting <type 'int'>, got <type 'str'>

Caution! If you use equal sign, the type consistency can be broken

In [23]:
typed_var = TypeVar(int,3)
typed_var = "str" # this WILL NOT work. NEVER DO THIS
typed_var

'str'

In [24]:
typed_var = TypeVar(int,3)
typed_var.assign("hello") # this is the correct way of assigning values, so errors can be found
typed_var

TypeError: Assignment failed. expecting <type 'int'>, got <type 'str'>

## 3. functions

You can still use **Typed** decorator for functions outside of classes

In [25]:
@Typed
def typed_func(a,b,c,d=1,types={'b':int,'c':int,'d':int}):
    # we do NOT need to specify all the types of parameters.
    # parameter 'a' can be of any types
    # type for 'return' is not necessary if you do not need
    # you can always write other default parameters, like d
    return a + b + c + d

In [26]:
typed_func(1,2,3) # a=1,b=2,c=3,d=1

7

In [27]:
typed_func(1,2,3,4) # a=1,b=2,c=3,d=4

10

In [28]:
typed_func("1",2,3) # error

TypeError: cannot concatenate 'str' and 'int' objects

## 4. Others

If you do not want to use **Typed** decorator, or **TypeBase**, just use **TypeVar** is fine

In [29]:
class Employee(object):
    
    total_number = TypeVar(int,0) # use TypeVar instance
    
    def __init__(self,name,sex,age,salary):
        self.name = TypeVar(str,name) # use TypeVar instance
        self.sex = TypeVar(str,sex) # use TypeVar instance
        self.age = TypeVar(int,age) # use TypeVar instance
        self.salary = TypeVar(int,salary) # use TypeVar instance
        Employee.total_number += 1
        
    def greet(self,words):
        print "{} says: {}".format(self.name,words)

In [30]:
mick = Employee('Mick','male',21,2000) # it will pass

In [31]:
someone = Employee(123,'male','21',2000) # it won't

TypeError: expected the type of 'value' to be <type 'str'>, got <type 'int'>

In [32]:
words = TypeVar.var("Nice to see ya") # use TypeVar instance
mick.greet(words)
words.assign("Good bye")
mick.greet(words)

Mick says: Nice to see ya
Mick says: Good bye
